<a href="https://colab.research.google.com/github/Richardxiao123/TW_stock_analysis/blob/main/%E8%82%A1%E7%A5%A8%E8%B3%87%E6%96%99%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler#目的：標準化數據
print('/n')

/n


In [ ]:

# 獲取股票數據
ticker = '2360.TW'
data = yf.download(ticker,start='2024-2-19', end="2024-3-7")

print(data, '\n', '--------------------------------------------------------')

# 進行數據標準化
scaler = MinMaxScaler()
data_temp = pd.DataFrame(index=data.index)  # 設定索引

data_temp['score'] = 2  # 設定分數

for a in ['Close', 'High', 'Low', 'Open', 'Volume']:
    data_temp[a] = scaler.fit_transform(data[[a]])  # 線性變換 0~1
print(data_temp)

# 10 天為一組轉到同一排
new_data = {}
new_data["score"] = [data_temp["score"].iloc[0]]

for a in ['Close', 'High', 'Low', 'Open']:
    for i in range(10):
        new_data[f"{a}{i+1}"] = data_temp[a].iloc[i]

# 轉換為 DataFrame
new_data = pd.DataFrame(new_data)

# 把 10 天股價展開後加到 data_train
data_test = pd.concat([data_test, new_data], ignore_index=True)

# 輸出轉換後的數據
print(data_test)


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open   Volume
Ticker         2360.TW     2360.TW     2360.TW     2360.TW  2360.TW
Date                                                               
2024-02-19  207.753754  211.175580  205.798424  207.753754  1866622
2024-02-20  217.041565  219.485727  208.242583  208.242583  3188328
2024-02-21  222.418716  228.284705  215.575064  218.019225  5555451
2024-02-22  226.329376  233.173029  224.862879  227.795873  5132166
2024-02-23  233.661865  245.393842  229.751206  230.240039  8530106
2024-02-26  231.706543  240.505526  231.217711  237.083699  4661520
2024-02-27  224.374054  231.706539  219.974563  220.463395  6120167
2024-02-29  227.795883  230.728877  224.862889  227.307051  3652052
2024-03-01  228.773544  234.150700  228.773544  230.240041  2103267
2024-03-04  238.550186  238.550186  228.284707  228.284707  3097880
2024-03-05  259.081146  261.036476  238.550187  241.483182  8650794
2024-03-06  268.857788  276.679106  247.837997  

In [ ]:
from google.colab import files
data_test.to_csv('data_test_1000.csv')
files.download('data_test_1000.csv')

In [ ]:
data_train =pd.read_csv("data_train.csv")
print(data_train)


In [ ]:
data_train['score'][6]=3

print(data_train)

In [ ]:
data_test=pd.DataFrame()

In [ ]:
import yfinance as yf

buy_list = ['1615.TW', '2059.TW', '2328.TW', '2379.TW', '2385.TW', '2387.TW', '6799.TW',
            '2392.TW', '2420.TW', '2455.TW', '2482.TW', '2491.TW', '2543.TW', '2603.TW',
            '2637.TW', '3004.TW', '3028.TW', '3501.TW', '3563.TW', '3588.TW', '3592.TW',
            '4562.TW', '4588.TW', '4912.TW', '4994.TW', '5269.TW', '6165.TW', '6176.TW',
            '6177.TW', '6196.TW', '6409.TW', '6412.TW', '6438.TW', '6491.TW', '6754.TW']

change_rate=[]
for ticker in buy_list:
        back_test = yf.download(ticker, start='2024-3-6', end="2024-3-7")
        back_test2 = yf.download(ticker, start='2024-3-21', end="2024-3-22")

        # 確認資料下載成功

        x = back_test['Close'].iloc[0].item()
        y = back_test2['Close'].iloc[0].item()
        z = y - x
        change_rate.append(z/x)
        print(f"{ticker}: 變動比例 = {z / x}")


print(change_rate)
print('報酬率',sum(change_rate))
amount = len(change_rate)
print('平均報酬率',sum(change_rate)/amount)

In [ ]:
import pandas as pd
import numpy as np


def augment_data(data: pd.DataFrame, target_size: int, noise_level: float = 0.02) -> pd.DataFrame:

    num_new_samples = target_size - len(data)
    np.random.seed(0)

    # 隨機從原始資料中選擇資料來進行增強
    selected_indices = np.random.choice(data.index, num_new_samples, replace=True)
    new_samples = data.iloc[selected_indices].copy()

    # 對數據進行增強
    for col in data.columns[3:]:  # 忽略 'Unnamed: 0', 'unnamed', 'score'
        original_values = new_samples[col].values
        noise = np.random.normal(0, noise_level, original_values.shape)

        # 隨機選擇 Scaling 或 Shifting
        random_choice = np.random.rand(len(original_values))
        adjusted_values = np.where(
            random_choice < 0.5,
            original_values * (1 + noise),  # 微調 (Scaling)
            original_values + noise  # 加入噪音 (Shifting)
        )

        new_samples[col] = adjusted_values

    # 將原始資料與增強資料合併
    augmented_data = pd.concat([data, new_samples], ignore_index=True)
    return augmented_data
